In [5]:
!pip install pandas simplekml requests


     ---------------------------------------- 53.0/53.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65874 sha256=55744a5d91b025e2abc4ef1eaefea5056a20277b7246b67e454a4dfe5470df4b
  Stored in directory: c:\users\johan stephen\appdata\local\pip\cache\wheels\c7\52\10\6762e569ce37094b4dd96234a8858c74d1ab2843251f96f86b
Successfully built simplekml


In [12]:
import pandas as pd
import requests
import simplekml
import math
import io

USGS_CSV = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.csv"


In [13]:
r = requests.get(USGS_CSV)
r.raise_for_status()

# wrap the raw text in io.StringIO, not pandas.compat.StringIO
df = pd.read_csv(io.StringIO(r.text))

df.head(5)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-05-17T02:20:20.120Z,37.477665,-118.535500,12.02,1.99,md,17.0,187.0,0.09748,0.06,...,2025-05-17T02:21:57.637Z,"9 km NE of Round Valley, CA",earthquake,0.55,1.27,0.180,17.0,automatic,nc,nc
1,2025-05-17T02:07:34.810Z,38.810665,-122.822334,3.03,0.31,md,9.0,75.0,0.00564,0.02,...,2025-05-17T02:27:17.179Z,"7 km NW of The Geysers, CA",earthquake,0.50,1.73,0.120,9.0,automatic,nc,nc
2,2025-05-17T02:03:31.570Z,33.971333,-116.824000,16.24,0.78,ml,23.0,54.0,0.06507,0.18,...,2025-05-17T02:07:02.926Z,"7 km NNW of Cabazon, CA",earthquake,0.33,0.71,0.173,17.0,automatic,ci,ci
3,2025-05-17T01:37:15.600Z,36.791832,-121.391335,5.27,1.22,md,8.0,123.0,0.05276,0.04,...,2025-05-17T01:52:15.949Z,"3 km SW of Ridgemark, CA",earthquake,0.37,1.05,0.120,7.0,automatic,nc,nc
4,2025-05-17T01:31:48.466Z,60.774200,-150.996600,22.20,1.30,ml,NaN,NaN,NaN,0.37,...,2025-05-17T01:33:41.485Z,"18 km ENE of Nikiski, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak


In [15]:
def make_circle(lat, lon, radius_km, segments=36):
    """Return list of (lon,lat) points approximating a circle."""
    coords = []
    for i in range(segments):
        angle = 2 * math.pi * i / segments
        # Offset in degrees approx (1° ≈ 111 km)
        dlat = (radius_km * math.cos(angle)) / 111.0
        dlon = (radius_km * math.sin(angle)) / (111.0 * math.cos(math.radians(lat)))
        coords.append((lon + dlon, lat + dlat))
    return coords

kml = simplekml.Kml()
for _, row in df.iterrows():
    lat, lon, mag = row.latitude, row.longitude, row.mag
    # choose a radius, e.g. 10 km per magnitude point
    radius = max(mag, 0) * 10  
    # outer ring
    pol = kml.newpolygon(name=f"M{mag:.1f}",
                         outerboundaryis=make_circle(lat, lon, radius))
    pol.style.polystyle.color = "40ff0000"  # translucent red
    # center point
    kml.newpoint(name=f"M{mag:.1f}", coords=[(lon, lat)])
kml.save("all_day_quakes_with_rings.kml")
